# Tailoring metabolic networks (future release)

In [3]:
import straindesign as sd
import cobra

cobra.Configuration().solver = 'cplex'
ijn = cobra.io.load_model('iJN746') # or  iJN1463

# exchange_reacs = [r for r in ijn.reactions if all(s<0 for s in r.metabolites.values())]
# # shut all exchange fluxes
# for r in exchange_reacs:
#     r.upper_bound = 0.0

# # shut CO2 uptake
# ijn.reactions.EX_co2_e.lower_bound    = 0.0

# # keep main fermentation products open
# ijn.reactions.EX_ac_e.upper_bound     = 1000.0
# ijn.reactions.EX_co2_e.upper_bound    = 1000.0
# ijn.reactions.EX_etoh_e.upper_bound   = 1000.0
# ijn.reactions.EX_for_e.upper_bound    = 1000.0
# ijn.reactions.EX_h2o2_e.upper_bound   = 1000.0
# ijn.reactions.EX_h2o_e.upper_bound    = 1000.0
# ijn.reactions.EX_h_e.upper_bound      = 1000.0
# ijn.reactions.EX_lac__D_e.upper_bound = 1000.0
# ijn.reactions.EX_meoh_e.upper_bound   = 1000.0
# ijn.reactions.EX_o2_e.upper_bound     = 1000.0
# ijn.reactions.EX_succ_e.upper_bound   = 1000.0
# ijn.reactions.EX_tungs_e.upper_bound  = 1000.0

# for r in ijn.reactions:
#     if 'DM_' in r.id:
#         r.upper_bound = 1000.0
        
atps_genes = [g for g in ijn.reactions.ATPS4rpp.genes]
ijn.reactions.ATPS4rpp.gene_reaction_rule = str(atps_genes[0])
cobra.manipulation.remove_genes(ijn,atps_genes[1:])
atps_genes[0].name = 'atpX'

min_lac_yield = 0.2
min_growth = 0.05
module_suppress = sd.SDModule(ijn,sd.names.SUPPRESS,constraints=f'EX_lac__D_e + {min_lac_yield} EX_glc__D_e <= 0')
module_protect  = sd.SDModule(ijn,sd.names.PROTECT, constraints=f'BIOMASS_KT2440_Core2>={min_growth}')

sd.plot_flux_space(ijn, ('BIOMASS_KT2440_Core2',('EX_lac__D_e','-EX_glc__D_e')));

import logging
logging.basicConfig(level=logging.INFO)

# allow all gene knockouts except for spontanuos 
gko_cost = {g.id:1 for g in ijn.genes}    
# possible knockout of O2
ko_cost = {'EX_o2_e': 1}
# Compute strain designs
sols = sd.compute_strain_designs(ijn,
                                 sd_modules = [module_suppress, module_protect],
                                 max_solutions = 1,
                                 max_cost = 40,
                                 ko_cost = ko_cost,
                                 gko_cost = gko_cost,
                                 solution_approach = sd.names.ANY)
# Print solutions
print(f"One compressed solution with cost {sols.sd_cost[0]} found and "+\
      f"expanded to {len(sols.reaction_sd)} solutions in the uncompressed netork.")
print(f"Example intervention set: {['+'+s if v>0 else '-'+s for s,v in sols.gene_sd[0].items() if v!=0]}")


import matplotlib.pyplot as plt
# Wild-type plot
datapoints, triang, plot1 = sd.plot_flux_space(ijn,
                                               ('BIOMASS_KT2440_Core2',('EX_lac__D_e','-EX_glc__D_e')),
                                               show=False);
_,          _,      plot2 = sd.plot_flux_space(ijn,
                                               ('BIOMASS_KT2440_Core2',('EX_lac__D_e','-EX_glc__D_e')),
                                               constraints=f'BIOMASS_KT2440_Core2>={min_growth}',
                                               show=False);
plot2.set_facecolor('#70AD47')
plot2.set_edgecolor('#70AD47')
# pGCP design plot
_,          _,      plot3 = sd.plot_flux_space(ijn,
                                               ('BIOMASS_KT2440_Core2',('EX_lac__D_e','-EX_glc__D_e')),
                                               # The sign of the glucose exchange reaction is flipped since 
                                               # reaction is defined in the direction of secretion.
                                               constraints=f'EX_lac__D_e + {min_lac_yield} EX_glc__D_e <= 0',
                                               show=False);
plot3.set_facecolor('#ED7D31')
plot3.set_edgecolor('#ED7D31')
# plotting designed strain
interventions = [[{s:1.0},'=',0.0] for s,v in sols.reaction_sd[0].items() if v < 1]
_,          _,      plot4 = sd.plot_flux_space(ijn,
                                               ('BIOMASS_KT2440_Core2',('EX_lac__D_e','-EX_glc__D_e')),
                                               # The sign of the glucose exchange reaction is flipped since 
                                               # reaction is defined in the direction of secretion.
                                               constraints=interventions,
                                               show=False);
plot4.set_facecolor('#FFC000')
plot4.set_edgecolor('#FFC000')
# adjust axes limits and show plot
plot4.axes.set_xlim(0, 1.05*max([a[0] for a in datapoints]))
plot4.axes.set_ylim(0, 1.05*max([a[1] for a in datapoints]))
plt.show()

Exception: Expression invalid. Unknown identifier BIOMASS_KT2440_Core2.

Continued ..

## Carving

## Gap-filling